In [1]:
import pandas as pd
import numpy as np
import re
import unidecode
import chardet
import bz2
import gc

In [2]:
#with open('data/training.1600000.processed.noemoticon.csv', 'rb') as f:
#    enc = chardet.detect(f.read())

In [3]:
#enc['encoding']

In [4]:
df = pd.read_csv('../data/training.1600000.processed.noemoticon.csv', header=None, encoding='latin')

In [5]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
columns = ['polarity', 'id', 'date', 'query', 'user', 'text']
df.columns = columns

In [7]:
df.drop(['id', 'date', 'query', 'user'], axis=1, inplace=True)

In [8]:
pol_switch = {'0': 0, '4': 1}

In [9]:
df['polarity'] = df['polarity'].apply(lambda x: pol_switch[str(x)])

In [10]:
def clean(text):
    t = text.lower()
    t = t.replace('\\n', ' ').replace('\\t', ' ').replace('\t', ' ').replace('. com', '.com')
    t = re.sub(r'https?:\/\/[a-z.\/A-Z\d]*', ' ', t)
    t = re.sub(r"\ [A-Za-z]*\.com", ' ', t)
    t = re.sub(r"@\S+", '', t)
    t = t.replace('@', '')
    t = unidecode.unidecode(t)
    t = t.replace('#', '_')
    
    to_replace = ["&quot;", ':&lt;', ':&gt;', '&amp;', '-&lt;', '-&gt;', '=&lt;', '=&gt;', 's&lt;', 's&gt;']
    for x in to_replace:
        t = t.replace(x, '')
    pattern = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    t = pattern.sub(r"\1\1", t)
    
    pattern = re.compile(r"([\s.,\/#!$%^&*?;:{}=_`()+-])\1{1,}")
    t = pattern.sub(r'\1', t)
    t = re.sub(' {2,}', '', t)
    t = t.lower()
    t = t.strip()
    t = t.rstrip()
    return t

In [11]:
t = "@Hello is going to watch ?Ai again, on her handphone during lit camp #tramp"
clean(t)

'is going to watch ?ai again, on her handphone during lit camp _tramp'

In [12]:
df['text'] = df['text'].apply(lambda x: clean(str(x)))

In [13]:
df['len'] = df['text'].apply(lambda x: len(x))

In [14]:
df.drop(df.loc[df['len'] == 0].index, axis=0, inplace=True)

In [15]:
df.drop('len', axis=1, inplace=True)

In [16]:
df.head()

,polarity,text
0,0,"- aww, that's a bummer. you shoulda got david ..."
1,0,is upset that he can't update his facebook by ...
2,0,i dived many times for the ball. managed to sa...
3,0,my whole body feels itchy and like its on fire
4,0,"no, it's not behaving at all. i'm mad. why am ..."


In [17]:
df['text'] = df['text'].astype('string')

In [18]:
df.head()

,polarity,text
0,0,"- aww, that's a bummer. you shoulda got david ..."
1,0,is upset that he can't update his facebook by ...
2,0,i dived many times for the ball. managed to sa...
3,0,my whole body feels itchy and like its on fire
4,0,"no, it's not behaving at all. i'm mad. why am ..."


In [19]:
df = df.sample(frac=1)

In [20]:
df.to_csv('../data/all.csv', index=None)

In [21]:
texts = list(df['text'])

In [22]:
texts[1152]

'renee showed odette off to dad today. he seemed a bit creeped out so glad she loves it, had a blast making her.'

In [23]:
valid_size = int(0.2*len(df))
df_train = df[:-valid_size]
df_valid = df[-valid_size:]

In [24]:
df_train.to_csv('../data/train.csv', index=None)
df_valid.to_csv('../data/valid.csv', index=None)